# Interpretability

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Titanic Datensatz

Die Spalten haben folgende Bedeutung:

Variable	|Definition	|Key
:---|:---|---
Survival	|Survival - Label	|0 = No, 1 = Yes
Pclass	|Ticket class	|1 = 1st, 2 = 2nd, 3 = 3rd
Sex	|Sex	| male, female
Age	|Age in years|	
SibSp	|# of siblings / spouses aboard the Titanic	|
Parch	| # of parents / children aboard the Titanic	|
Ticket	|Ticket number	|
Fare	|Passenger fare	|
Cabin	|Cabin number	|
Embarked	|Port of Embarkation	|C = Cherbourg, Q = Queenstown, S = Southampton
Name|Name des Passagiers |


In [ ]:
train = pd.read_csv('./data/titanic/train.csv')
test = pd.read_csv('./data/titanic/test.csv')

## Datenaufbereitung

In [ ]:
train.drop(['Cabin'], axis=1, inplace=True)
train.info();

In [ ]:
age_group = train.groupby("Pclass")["Age"]
print(age_group.median())

Der Altersmedian unterscheided sich zwischen den Klassen signifikant, also setzen wir diesen für die fehlenden Werte ein:

In [ ]:
train.loc[train.Age.isnull(), 'Age'] = train.groupby("Pclass").Age.transform('median')
print(train["Age"].isnull().sum())

In [ ]:
train.drop(['PassengerId', 'Name', 'Ticket', 'Fare'], axis = 1, inplace = True)
train["Embarked"] = train["Embarked"].fillna('S')
train.isnull().sum()

In [ ]:
train_onehot = pd.get_dummies(train, drop_first=True)
train_onehot

## Training vorbereiten

`train` enthält ja sowohl unsere Features als auch das Label `Survived`. Dies teilen wir jetzt auf in X und y:

In [ ]:
y = train_onehot['Survived']
X = train_onehot.drop('Survived', axis=1)
X, y

Schließlich splitten wir unsere Daten noch in ein Training- und ein Test-Set im Verhältnis 80:20.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Logistische Regression

Trainiere das Modell

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

... und bewerte das Ergebnis ...

**Confusion Matrix:** Die Confusion-Matrix $C$ ist definiert durch: $C_{i,j}$ ist die Anzahl der Beobachtungen der wahren Gruppe $i$, die als zur Gruppe $j$ vorhergesagt werden.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

predictions = model.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print("\t'false'\t'true'")
print("false\t  {}\t  {}".format(tn, fp))
print("true\t  {}\t  {}".format(fn, tp))

### Kennzahlen zur Bewertung

**Precision** ist das Verhältnis $\frac{tp}{tp + fp}$, wobei $tp$ die Anzahl der echten positiven und $fp$ die Anzahl der falschen positiven Werte ist. Intuitiv ist Precision die Fähigkeit des Klassifikators, eine negative Probe nicht als positiv zu kennzeichnen.

**Recall** ist das Verhältnis $\frac{tp}{tp + fn}$, wobei $fn$ die Anzahl der falschen Negativen ist. Intuitiv ist Recall die Fähigkeit des Klassifikators, alle positiven Proben zu finden.

Der **F-Beta-Score** kann als ein gewichteter harmonischer Mittelwert von Precision und Recall interpretiert werden, wobei ein F-Beta-Score seinen besten Wert bei 1 und den schlechtesten Wert bei 0 erreicht.

$f1=2\times\frac{Precision \times Recall}{Precision + Recall}$

Der F-Beta-Score gewichtet um den Faktor Beta Recall mehr als die Precision. Beta == 1,0 bedeutet, dass Recall und Precision gleich wichtig sind.

**Support** ist die jeweilige Anzahl der Vorkommnisse der wahren Labels.

In [ ]:
print(classification_report(y_test, predictions, target_names=['Nicht überlebt', 'Überlebt']))

### Interpretation des Modells

Schauen wir uns Intercept und Koeffizienten an

In [ ]:
print('Intercept/Bias: {}'.format(model.intercept_))

coef_dict = sorted(list(zip(X.columns.tolist(), model.coef_.ravel())), key=lambda tup: tup[1])
for tup in coef_dict:
    print(tup)

Das Geschlecht ist mit Abstand der am stärksten eingehende Faktor, gefolgt von der Klasse.

Da wir die Features nicht skaliert haben, stimmt die Reihenfolge bzgl SibSp, Parch und insbesondere Age so allerdings nicht.

In [ ]:
X['Age'].mean()

d.h. Age geht eigentlich mit einem fast 30-fachen Gewicht gegenüber den 0/1-Features ein und steht damit eher an dritter Stelle.

SibSp und Parch könnte man vielleicht als Familie zusammenfassen (Familiengröße = 1 + SibSp + Parch) und dann trainieren, oder eventuell die Familiengröße auch One-Hot Encoden.

Das geht dann in Richtung "Feature Engineering". Ein interessanter Artikel dazu im Kontext des Titanic Datasets 
ist [hier](https://medium.com/i-like-big-data-and-i-cannot-lie/how-i-scored-in-the-top-9-of-kaggles-titanic-machine-learning-challenge-243b5f45c8e9) zu finden.


## LIME



In [ ]:
import lime
import lime.lime_tabular

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(train_onehot, 
                                                   feature_names=['Pclass', 'Age', 'SibSp', 'Parch', 'Sex_male', 'Embarked_Q', 'Embarked_S'], 
                                                   class_names=[0, 1], 
                                                   categorical_features=[0, 4, 5, 6], 
                                                   verbose=True, 
                                                   mode='regression')